In [1]:
import csv
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
from scipy.optimize import root, fsolve
from scipy.special import binom
import pandas as pd

In [30]:
# Initial data
T_idle = 3
T_rts = 3
T_cts = 3
T_ack = 3
T_bo = 3
T_data = 3
T_wait = 3
T_out = 3

T_max = 12
Nr = 3
p_a = 1
nodes = 20

### Formulas for exponential backoff

In [31]:
def Etc(p, p_f):
    if p_f == 1.:
        p_f = 0.9999999
    elif p_f == 0.:
        p_f = 0.0000001
    total_sum = 0
    for i in range(1, Nr + 1 + 1):
        first_addition = T_wait + T_bo / (1 - p_f)
        numerator = p_f ** (2**i * T_max + 1) - 2**i * T_max * p_f - p_f + 2**i * T_max
        denominator = p_f * (1 - p_f ** (2**i * T_max))
        second_addition = (1 - p) * (T_cts + T_data + T_ack)
        huge_sum = T_rts + p * T_out / p_f + first_addition * numerator / denominator + second_addition
        total_sum += p ** (i - 1) * huge_sum
    return T_idle + p_a * total_sum

In [32]:
pi_rts(0.4, 0.8)

0.01575801677131626

In [33]:
pi_rts(0.4, 0.8)

0.01575801677131626

In [34]:
def pi_idle(p, p_f):
    return T_idle / Etc(p, p_f)

def pi_rts(p, p_f):
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        numerator = p ** (i - 1) * p_a * T_rts
        denominator = Etc(p, p_f)
        total_sum += numerator / denominator
    return total_sum

def pi_out(p, p_f):
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        numerator = p ** (i - 1) * p_a * p * T_out / p_f
        denominator = Etc(p, p_f)
        total_sum += numerator / denominator
    return total_sum

def pi_cts(p, p_f):
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        numerator = (1 - p) * p ** (i - 1) * p_a * T_cts
        denominator = Etc(p, p_f)
        total_sum += numerator / denominator
    return total_sum

def pi_bo(p, p_f):
    if p_f == 1.:
        p_f = 0.9999999
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        numerator = p ** (i - 1) * p_a * T_bo * (p_f ** (2**i * T_max + 1) - 2**i * T_max * p_f - p_f + 2**i * T_max)
        denominator = Etc(p, p_f) * (1 - p_f) * p_f * (1 - p_f ** (2**i * T_max))
        total_sum += numerator / denominator
    return total_sum

def pi_wait(p, p_f):
    if p_f == 1.:
        p_f = 0.9999999
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        numerator = p ** (i - 1) * p_a * T_wait * (p_f ** (2**i * T_max + 1) - 2**i * T_max * p_f - p_f + 2**i * T_max)
        denominator = Etc(p, p_f) * p_f * (1 - p_f ** (2**i * T_max))
        total_sum += numerator / denominator
    return total_sum

def pi_data(p, p_f):
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        numerator = (1 - p) * p ** (i - 1) * p_a * T_data
        denominator = Etc(p, p_f)
        total_sum += numerator / denominator
    return total_sum

def pi_ack(p, p_f):
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        numerator = (1 - p) * p ** (i - 1) * p_a * T_ack
        denominator = Etc(p, p_f)
        total_sum += numerator / denominator
    return total_sum

In [35]:
def p_success(p):
    total_sum = 0
    for i in range(1, Nr + 1 + 1):
        total_sum += (1 - p) * p ** (i - 1) * p_a
    return total_sum

# alternative formula
def p_success(p):
    return 1 - p ** (Nr + 1)

def p_failure(p):
    return p ** (Nr + 1)

In [36]:
def transcendental_equation_p_collision(p, p_f, n):
    p_rts = pi_rts(p, p_f)
    p_cts = pi_cts(p, p_f)
    p_data = pi_data(p, p_f)
    p_ack = pi_ack(p, p_f)
    # We solve transcendental equation f(x) = g(x) by turning it into f(x) - g(x) = 0
#     return p - (1 - (1 - p_rts) ** (n) - n * p_rts * (1 - p_rts) ** (n-1))
#     return p - (1 - (1 - p_rts) ** (2*(n-1))) - (1 - (1 - p_cts) ** (2*(n-1)))
    return p - (1 - (1 - p_rts) ** ((n-1))) - (1 - (1 - p_cts) ** ((n-1)))

def transcendental_equation_p_free(p, p_f, n):
    p_rts = pi_rts(p, p_f)
    p_cts = pi_cts(p, p_f)
    p_data = pi_data(p, p_f)
    p_ack = pi_ack(p, p_f)
    return p_f - (1 - p_cts) ** (n-1)

def system_of_equations(p_pf, node):
    p, pf = p_pf
    return (transcendental_equation_p_collision(p, pf, node), transcendental_equation_p_free(p, pf, node))

def calculate_p_pf_from_system():
    p_array = []
    pf_array = []
    for node in range(1, nodes+1):
        p, pf = fsolve(system_of_equations, (0.01, 0.01), args=(node))
        p_array.append(p)
        pf_array.append(pf)
    return p_array, pf_array

In [37]:
p_array, p_f_array = calculate_p_pf_from_system()

In [38]:
p_array

[1.8086627663754192e-27,
 0.08529327098433073,
 0.13767085223213002,
 0.17633124590459104,
 0.2074630534888145,
 0.2337707762791545,
 0.2566901124591728,
 0.2770822024033302,
 0.29550795106873645,
 0.31235497654279315,
 0.3279033380583719,
 0.342362529527237,
 0.35589368543594385,
 0.36862361000159477,
 0.3806540172380889,
 0.39206783067700757,
 0.4029336053953271,
 0.4133087103281595,
 0.42324166833504717,
 0.4327739096729536]

In [39]:
p_f_array

[1.0,
 0.9592531181267471,
 0.9361629239627239,
 0.9201005510630557,
 0.907791616952396,
 0.8978376140253885,
 0.8895085152051194,
 0.8823726239977049,
 0.8761523035325373,
 0.870657912005629,
 0.8657539051611225,
 0.8613399134477483,
 0.8573394698028102,
 0.8536929395359573,
 0.8503528958391291,
 0.8472809877707084,
 0.8444457557883412,
 0.8418210693682319,
 0.839384984958541,
 0.8371188951308496]

In [40]:
prt = pi_rts(p_array[9], p_f_array[9])
prt

0.022208778449657764

In [54]:
1 - (1 - prt) ** (10) - 10 * prt * (1 - prt) ** (9)

0.022323427120735456

In [55]:
(1 - prt) ** (10)

0.786510355786292

In [56]:
10 * prt * (1 - prt) ** (9)

0.19116621709297252

In [41]:
analytics_headers = [
    'nodes',
    'p_collision',
    'p_success',
    'p_failure',
    'p_free',
    'cycle_time',
    'p_bo',
    'p_wait',
    'p_rts',
    'p_out',
    'p_cts',
    'p_data',
    'p_ack',
    'bo_time',
    'wait_time',
    'rts_time',
    'out_time',
    'cts_time',
    'data_time',
    'ack_time',
]

In [42]:
data = [analytics_headers]
for i in range(nodes):
    data.append([
        i + 1,
        p_array[i],
        p_success(p_array[i]),
        p_failure(p_array[i]),
        p_f_array[i],
        Etc(p_array[i], p_f_array[i]),
        pi_bo(p_array[i], p_f_array[i]),
        pi_wait(p_array[i], p_f_array[i]),
        pi_rts(p_array[i], p_f_array[i]),
        pi_out(p_array[i], p_f_array[i]),
        pi_cts(p_array[i], p_f_array[i]),
        pi_data(p_array[i], p_f_array[i]),
        pi_ack(p_array[i], p_f_array[i]),
        pi_bo(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_wait(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_rts(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_out(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_cts(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_data(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_ack(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
    ])

In [43]:
# Draw table to evaluate the results
data_pd = {}
for i in range(1, len(data[0])):
    data_pd[data[0][i]] = np.array(data[1:]).T[i]
df1 = pd.DataFrame(data_pd, index=np.arange(1, len(np.array(data[1:]).T[0])+1))
df1

,p_collision,p_success,p_failure,p_free,cycle_time,p_bo,p_wait,p_rts,p_out,p_cts,p_data,p_ack,bo_time,wait_time,rts_time,out_time,cts_time,data_time,ack_time
1,1.808663e-27,1.000000,1.070115e-107,1.000000,52.525589,0.714425,7.144248e-08,0.057115,1.033018e-28,0.057115,0.057115,0.057115,37.525585,0.000004,3.000000,5.425988e-27,3.000000,3.000000,3.000000
2,8.529327e-02,0.999947,5.292478e-05,0.959253,73.621369,0.757632,3.087114e-02,0.044546,3.960902e-03,0.040747,0.040747,0.040747,55.777897,2.272775,3.279566,2.916070e-01,2.999841,2.999841,2.999841
3,1.376709e-01,0.999641,3.592262e-04,0.936163,92.431317,0.777423,4.962839e-02,0.037625,5.533037e-03,0.032445,0.032445,0.032445,71.858206,4.587218,3.477700,5.114259e-01,2.998922,2.998922,2.998922
4,1.763312e-01,0.999033,9.667565e-04,0.920101,109.480127,0.787911,6.295366e-02,0.033236,6.369529e-03,0.027376,0.027376,0.027376,86.260597,6.892174,3.638720,6.973368e-01,2.997100,2.997100,2.997100
5,2.074631e-01,0.998147,1.852521e-03,0.907792,125.316749,0.794112,7.322375e-02,0.030150,6.890360e-03,0.023895,0.023895,0.023895,99.515483,9.176162,3.778300,8.634775e-01,2.994442,2.994442,2.994442
6,2.337708e-01,0.997014,2.986489e-03,0.897838,140.275596,0.798042,8.152990e-02,0.027828,7.245592e-03,0.021323,0.021323,0.021323,111.945854,11.436656,3.903585,1.016380e+00,2.991041,2.991041,2.991041
7,2.566901e-01,0.995659,4.341468e-03,0.889509,154.563811,0.800649,8.846486e-02,0.025999,7.502620e-03,0.019325,0.019325,0.019325,123.751306,13.673466,4.018480,1.159634e+00,2.986976,2.986976,2.986976
8,2.770822e-01,0.994106,5.894331e-03,0.882373,168.317398,0.802428,9.438747e-02,0.024510,7.696486e-03,0.017718,0.017718,0.017718,135.062552,15.887054,4.125389,1.295453e+00,2.982317,2.982317,2.982317
9,2.955080e-01,0.992374,7.625647e-03,0.876152,181.630491,0.803664,9.953192e-02,0.023267,7.847322e-03,0.016391,0.016391,0.016391,145.969863,18.078031,4.225914,1.425313e+00,2.977123,2.977123,2.977123
10,3.123550e-01,0.990481,9.519052e-03,0.870658,194.571132,0.804530,1.040596e-01,0.022209,7.967564e-03,0.015272,0.015272,0.015272,156.538360,20.246998,4.321187,1.550258e+00,2.971443,2.971443,2.971443


In [44]:
# Here we save our table to csv in the same folder
with open('2021-04-08 2.csv', "wt", newline="") as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(analytics_headers)
    for i in range(1, len(data)):
        writer.writerow(data[i])

In [17]:
s = 0
for i in range(1, 21):
    s += i
    print(s)

1
3
6
10
15
21
28
36
45
55
66
78
91
105
120
136
153
171
190
210


In [18]:
s

210

In [20]:
simpc = 0.5
simpf = 0.7
t1 = pi_idle(simpc, simpf) + pi_bo(simpc, simpf) + pi_rts(simpc, simpf)
t2 = pi_cts(simpc, simpf) + pi_data(simpc, simpf) + pi_ack(simpc, simpf)
t3 = pi_out(simpc, simpf) + pi_wait(simpc, simpf)
t1 + t2 + t3

0.9999813426418602

In [14]:
vc = 0.4731246508351256
vf = 0.8260981839009207
Etc(vc, vf)

362.2161884781131

In [29]:
s = 0
for i in range(9, 13):
    s += i
print(s)

42


In [30]:
s = 0
for i in range(13, 16):
    s += i
print(s)

42


In [31]:
s = 0
for i in range(16, 18):
    s += i
print(s)

33


In [32]:
s = 0
for i in range(18, 20):
    s += i
print(s)

37


In [33]:
s = 0
for i in range(20, 21):
    s += i
print(s)

20


In [ ]:
1 - 8
9 - 12
13 - 15
16 - 17
18 - 19
20